In [1]:
from pathlib import Path

import pandas as pd

from rtnls_fundusprep.utils import preprocess_for_inference

## Preprocessing

This code will preprocess the images and write .png files with the square fundus image and the contrast enhanced version

This step is not strictly necessary, but it is useful if you want to run the preprocessing step separately before model inference


Create a list of files to be preprocessed:

In [2]:
ds_path = Path("../samples/fundus")
files = list((ds_path / "original").glob("*"))

Images with .dcm extension will be read as dicom and the pixel_array will be read as RGB. All other images will be read using PIL's Image.open

In [3]:
bounds = preprocess_for_inference(
    files,  # List of image files
    rgb_path=ds_path / "rgb",  # Output path for RGB images
    ce_path=ds_path / "ce",  # Output path for Contrast Enhanced images
    n_jobs=4,  # number of preprocessing workers
)
df_bounds = pd.DataFrame(bounds).set_index("id")

0it [00:00, ?it/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
6it [00:00, 143.58it/s]
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:    2.1s remaining:    4.2s
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:    2.1s remaining:    2.1s
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:    2.9s remaining:    1.4s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    4.3s finished


The preprocessor will produce RGB and contrast-enhanced preprocessed images cropped to a square and return a dataframe with the image bounds that can be used to reconstruct the original image. Output files will be named the same as input images, but with .png extension. Be careful with providing multiple inputs with the same filename without extension as this will result in over-written images. Any exceptions during pre-processing will not stop execution but will print error. Images that failed pre-processing for any reason will be marked with `success=False` in the df_bounds dataframe.

In [4]:
df_bounds.to_csv(ds_path / "meta.csv")